In [27]:
import numpy as np
import scipy.special
import matplotlib.pyplot as plt
from tqdm import tqdm  # Import tqdm for progress bar

# neural network class definition
class NeuralNetwork:
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # set number of nodes in each input, hidden, output layer
        self.i_nodes = input_nodes
        self.h_nodes = hidden_nodes
        self.o_nodes = output_nodes
        # link weight matrices, input_hidden, hidden_output
        self.i_h_weight = np.random.normal(0.0, pow(self.h_nodes, -0.5), (self.h_nodes, self.i_nodes))
        self.h_o_weight = np.random.normal(0.0, pow(self.o_nodes, -0.5), (self.o_nodes, self.h_nodes))
        # learning rate
        self.lr = learning_rate
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)

    # train the neural network
    def train(self, inputs_list, targets_list):
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        
        # forward pass
        hidden_inputs = np.dot(self.i_h_weight, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        final_inputs = np.dot(self.h_o_weight, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)

        # error calculation
        output_errors = targets - final_outputs
        hidden_errors = np.dot(self.h_o_weight.T, output_errors)

        # update the weight
        self.h_o_weight += self.lr * np.dot((output_errors * final_outputs * (1.0 - final_outputs)), np.transpose(hidden_outputs))
        self.i_h_weight += self.lr * np.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), np.transpose(inputs))

    # query the neural network
    def query(self, inputs_list):
        inputs = np.array(inputs_list, ndmin=2).T
        hidden_inputs = np.dot(self.i_h_weight, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        final_inputs = np.dot(self.h_o_weight, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        return final_outputs


# number of input, hidden, output layer
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

# learning rate is 0.1
learning_rate = 0.1

# create neural network
n = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

# load mnist training data
train_dataPath = '/home/liy/CQU_EIE/dataEngineer/digitReco/data/mnist_train.csv'
with open(train_dataPath, 'r') as training_data_file:
    training_data_list = training_data_file.readlines()

# training the neural network
epochs = 1
for e in range(epochs):
    print(f"Epoch {e + 1}/{epochs}")
    for record in tqdm(training_data_list, desc="Training", ncols=100, bar_format="{l_bar}{bar} [time: {elapsed}]"):
        all_values_training = record.split(',')
        inputs = (np.asfarray(all_values_training[1:]) / 255.0 * 0.99) + 0.01
        targets = np.zeros(output_nodes) + 0.01
        targets[int(all_values_training[0])] = 0.99
        n.train(inputs, targets)

# test the neural network
# load the test data
test_dataPath = '/home/liy/CQU_EIE/dataEngineer/digitReco/data/mnist_test.csv'
with open(test_dataPath, 'r') as test_data_file:
    test_data_list = test_data_file.readlines()

# test
scorecard = []
for record in test_data_list:
    all_values_test = record.split(',')
    correct_label = int(all_values_test[0])
    inputs_test = (np.asfarray(all_values_test[1:]) / 255.0 * 0.99) + 0.01
    output_test = n.query(inputs_test)
    label = np.argmax(output_test)
    if label == correct_label:
        scorecard.append(1)
    else:
        scorecard.append(0)

# calculate the performance score
scorecard_array = np.asarray(scorecard)
print('performance = ', scorecard_array.sum() / scorecard_array.size)


Epoch 1/1


Training: 100%|███████████████████████████████████████████████████████████████████████ [time: 00:17]


performance =  0.9582
